# Training a network against data from the Burgers equation

Training a Convolutional Neural Network (CNN) with data from the Burgers equation. See model_1d folder for the code that generated the data.

A simple setup for a surrogate model predicts the next time-step from the current one. This all starts from a mathematical model, which tyically involves Patial Differential Equations, but here we just collect the dynamic variables in the model in a vector $x$ at discrete times $t_0,t_1,\ldots,t_K$ and denote these as $x_k$. We can represent the numerical model as:

$x_{k+1}=M(x_k)$, starting with given initial condition $x_0$

Next, we run the numerical model and collect $x_0,x_1,\ldots,x_K$ as training data for our Machine Learning (ML) model. The aim of the ML model will be to approximate the numerical model, and we'll call it a surrogate model. It's main purpose is to have a faster but slightly less accurate version of the model. We denote the surrogate model as: 

$\hat{x}_{k+1}=\hat{M}(\hat{x}_k,\theta)$

starting with $\hat{x}_0=x_0$, which is the true initial condition.

This model has $\hat{x}_k$ as input and $x_{k+1}$ as output, and $\theta$ is a vector of parameters of the ML model to be estimated. For a given vector of parameters, one can sequentially compute $\hat{x}_1$, $\hat{x}_2$, $\hat{x}_3$, etc. This is called a rollout.

## Training

For training the most basic approach is to use one-step-ahead predictions. This means that the target is to accurately approximate the output of the numerical model. We can write this as a loss function:

$J(\theta) = \sum_{k=1}^{K-1} | x_{k+1} - \hat{M}(x_k,\theta)|^2$

Viewed as a supervised learning method, we have the generated data $x_k$ as inputs and $x_{k+1}$ as outputs. As is common in such cases the input data are collected in an input array $X$ and an output array $Y$ that consist of:

$X=[x_0,x_1,\ldots,x_{K-1}]$
$Y=[x_1,x_2,\ldots,x_{K}]$ 

We can generalize $\hat{M}$ to matrices to obtain:

$Y \approx \hat{M}(X,\theta)$

Unfortunately there is no guarantee that reasonably accurate one-step-ahead predictions lead to accurate rollouts, since the errors can accumulate quickly.


In [1]:
# Load Packages

# swithch to the directory where this script is located
cd(@__DIR__)

# Packages
using Pkg
Pkg.activate(".")
#Pkg.instantiate()
using Flux
using BSON
using Plots, Measures
using JLD2

# optional gpu usage
const use_gpu = false
if use_gpu
    using CUDA
end

  Activating project at `~/src_nobackup/julia_ml_tests.jl.git/training_1d_flux`


In [2]:
# Load the data

input_file = joinpath("..", "model_1d", "burgers1d_periodic.jld2")
if !isfile(input_file)
    println("Data file not found: $input_file")
end
println("Loading data from $input_file")
data=load(input_file)

# Convert data to input and output arrays

n_times=length(data["solution"])
n_points=length(data["solution"][1])

# Create inputs X and outputs Y
# u is the solution, t is the time, x is the spatial coordinate
X = zeros(Float32,n_points,1,n_times-1) # Float32 for GPU compatibility, 1 channel since only one variable u
Y = zeros(Float32,n_points,1,n_times-1) 

for t in 1:n_times-1
    X[:,:,t] .= data["solution"][t]
    Y[:,:,t] .= data["solution"][t+1]
end

@show X, size(X)
@show Y, size(Y)

# some metadata for plotting
output_times = data["times"][2:end]
output_x = data["grid"]
x0= X[:,:,1] # initial condition for rollout
nsteps = length(output_times)
nothing

Loading data from ../model_1d/burgers1d_periodic.jld2
(X, size(X)) = (Float32[1.0000124; 1.0000204; 1.0000335; 1.0000552; 1.000091; 1.00015; 1.0002472; 1.0004075; 1.0006717; 1.0011073; 1.0018255; 1.0030092; 1.0049608; 1.0081781; 1.013482; 1.0222248; 1.0366325; 1.0603589; 1.0993551; 1.1631098; 1.265857; 1.4251736; 1.6481644; 1.8869752; 2.0002217; 1.8871337; 1.648501; 1.42573; 1.2667018; 1.1643479; 1.1011395; 1.0629106; 1.0402671; 1.027392; 1.0208215; 1.0185977; 1.0197495; 1.0239954; 1.0316011; 1.0433427; 1.060552; 1.0852244; 1.1201605; 1.1690493; 1.2362334; 1.3254858; 1.4364765; 1.5576814; 1.6591731; 1.6999348; 1.6591057; 1.5575349; 1.4362254; 1.3250866; 1.2356162; 1.1681058; 1.1187238; 1.0830386; 1.0572246; 1.0382719; 1.0238616; 1.0121629; 1.0016258; 0.9907839; 0.9780511; 0.9614916; 0.9385368; 0.90563315; 0.8578477; 0.78863484; 0.69051707; 0.55880505; 0.4022868; 0.26030886; 0.20022185; 0.26015037; 0.40195024; 0.5582487; 0.6896722; 0.7873968; 0.85606325; 0.9030815; 0.93490213; 0.9563241

In [ ]:
# check for number of available samples
println("Number of samples: $n_samples")

Number of samples: 80


In [35]:
# Settings

# training
n_epochs = 10
batch_size = 35
learning_rate = 0.001
n_train = 70 # number of training samples, set to 1000 for faster training
n_val = 10 # number of validation samples, set to 100 for faster training
# check if we have enough data
n_samples = size(X, 3)
if n_train + n_val > n_samples
    error("n_train + n_val must be less than or equal to the number of samples: $n_samples")
end

# model
n_in = 1 # number of input channels (X)
n_out = 1 # number of output channels (Y)
n_hidden = 16 # number of hidden features in the model
#n_layers = 3 # number of hidden layers in the model
n_filter= 3 # width of the convolutional filter

3

In [12]:
# Split the data into training and validation sets
X_train_cpu = X[:,:,1:n_train]
Y_train_cpu = Y[:,:,1:n_train]
X_val_cpu = X[:,:,n_train+1:n_train+n_val]
Y_val_cpu = Y[:,:,n_train+1:n_train+n_val]
@show size(X_train_cpu), size(Y_train_cpu)
@show size(X_val_cpu), size(Y_val_cpu)

(size(X_train_cpu), size(Y_train_cpu)) = ((100, 1, 70), (100, 1, 70))
(size(X_val_cpu), size(Y_val_cpu)) = ((100, 1, 10), (100, 1, 10))


((100, 1, 10), (100, 1, 10))

In [ ]:
# move data to gpu
if use_gpu
    X_train = gpu(X_train_cpu)
    Y_train = gpu(Y_train_cpu)
    X_val = gpu(X_val_cpu)
    Y_val = gpu(Y_val_cpu)
else
    X_train = X_train_cpu
    Y_train = Y_train_cpu
    X_val = X_val_cpu
    Y_val = Y_val_cpu
end

# create the data loaders
train_loader = Flux.DataLoader((X_train, Y_train), batchsize=min(batch_size,n_train), shuffle=true)

val_loader = Flux.DataLoader((X_val, Y_val), batchsize=min(batch_size,n_val), shuffle=false)

@show train_loader
@show val_loader
nothing

train_loader = DataLoader(::Tuple{Array{Float32, 3}, Array{Float32, 3}}, shuffle=true, batchsize=35)
val_loader = DataLoader(::Tuple{Array{Float32, 3}, Array{Float32, 3}}, batchsize=10)


In [19]:
# Create the model
n_pad=n_filter ÷ 2 # padding for the convolutional layer
PadCircular(x) = pad_circular(x, (n_pad,n_pad,0,0,0,0))

model_cpu = Chain(
    PadCircular,
    Conv((n_filter,), n_in=>n_hidden, relu),
    PadCircular,
    Conv((n_filter,), n_hidden=>n_out)
)

if use_gpu
    model = gpu(model_cpu)
else
    model = model_cpu
end

Chain(
  Main.PadCircular,
  Conv((3,), 1 => 16, relu),            # 64 parameters
  Main.PadCircular,
  Conv((3,), 16 => 1),                  # 49 parameters
)                   # Total: 4 arrays, 113 parameters, 756 bytes.

In [30]:
# Create loss function

mse_loss(model,x, y) = Flux.mse(model(x), y)

mse_loss (generic function with 2 methods)

In [36]:

# Initialize the ADAM optimizer with default settings
optimizer = Flux.setup(Adam(learning_rate),model)

# Train the model
# and display the accuracy on each
# iteration
for epoch in 1:n_epochs
    # Train the model on the training data
    # and display the loss on eachiteration
	Flux.train!(mse_loss, model, train_loader, optimizer)
    println("epoch: $(epoch)") #NOTE: we careless with compute, rerunning model each time below
    println("   Loss for training    : $(mse_loss(model,X_train,Y_train))")
    println("   Loss for validation  : $(mse_loss(model,X_val,Y_val))")
end
     


epoch: 1
   Loss for training    : 0.022164416
   Loss for validation  : 0.02042228
epoch: 2
   Loss for training    : 0.01017732
   Loss for validation  : 0.009048727
epoch: 3
   Loss for training    : 0.0031879512
   Loss for validation  : 0.002495313
epoch: 4
   Loss for training    : 0.0006690051
   Loss for validation  : 0.00023962812
epoch: 5
   Loss for training    : 0.0013145433
   Loss for validation  : 0.0010068099
epoch: 6
   Loss for training    : 0.0032003522
   Loss for validation  : 0.0029261447
epoch: 7
   Loss for training    : 0.004583451
   Loss for validation  : 0.004309921
epoch: 8
   Loss for training    : 0.004690876
   Loss for validation  : 0.0044164904
epoch: 9
   Loss for training    : 0.0037271092
   Loss for validation  : 0.0034535006
epoch: 10
   Loss for training    : 0.0023578824
   Loss for validation  : 0.00207557


In [42]:
size(x0)

(100, 1)

In [43]:
# unroll the model to get the output for the initial condition
function unroll(model, x0, nsteps)
    x = reshape(x0, size(x0, 1), size(x0, 2), 1) # ensure x0 is 3
    outputs = zeros(Float32, size(x0, 1), size(x0, 2), nsteps+1)
    outputs[:,:,1] .= x0 # store the initial condition
    # unroll the model for nstep
    for t in 1:nsteps
        x = model(x)
        outputs[:,:,t+1] .= x
    end
    return outputs
end

# unroll the model to get the output for the initial condition
Y_unroll = unroll(model, x0, nsteps)

100×1×81 Array{Float32, 3}:
[:, :, 1] =
 1.0000124
 1.0000204
 1.0000335
 1.0000552
 1.000091
 1.00015
 1.0002472
 1.0004075
 1.0006717
 1.0011073
 ⋮
 0.99821854
 0.9988059
 0.99919957
 0.99946344
 0.99964035
 0.9997589
 0.9998384
 0.9998917
 0.9999274

[:, :, 2] =
 1.0451009
 1.0451441
 1.0451705
 1.0451889
 1.0452194
 1.04527
 1.0453532
 1.0454903
 1.0457165
 1.0460892
 ⋮
 1.0428531
 1.0436077
 1.0441134
 1.0444524
 1.0446794
 1.044832
 1.044934
 1.0450046
 1.0450602

[:, :, 3] =
 1.0878797
 1.087918
 1.0879514
 1.0879796
 1.0880084
 1.0880516
 1.0881226
 1.0882401
 1.0884337
 1.0887527
 ⋮
 1.0850022
 1.0859711
 1.0866208
 1.0870564
 1.0873482
 1.087544
 1.0876758
 1.0877662
 1.0878301

;;; … 

[:, :, 79] =
 1.8579173
 1.8575275
 1.8571253
 1.8567152
 1.8563035
 1.8558962
 1.8554993
 1.8551166
 1.8547552
 1.8544192
 ⋮
 1.8602054
 1.8600862
 1.8599317
 1.8597417
 1.8595164
 1.8592558
 1.8589625
 1.8586392
 1.85829

[:, :, 80] =
 1.8587377
 1.8583791
 1.8580065
 1.8576243
 1.8572373
 1

In [ ]:
# Make movie of the unrolled output
@time begin
    # create a movie of the solution
    anim = @animate for i in 1:length(sol.t)
        p1 = plot(x, sol[i], label="h(t=$(round(sol.t[i],digits=2)))", xlabel="x", ylabel="u", ylim=(0, 2))
        plot(p1, size=(800,400), title="Solution of the 1D Burgers equation")
    end
    gif(anim, "burgers1d_periodic.gif", fps=15)
end